# Example of aggregating multiple functions on the same columns of data

In [1]:
import numpy as np
import pandas as pd
import phjGenerateExampleData as phjData

In [2]:
phjTempDF = phjData.phjPandasDF12()

print(phjTempDF.dtypes)

# Could Convert date to datetime if it wasn't already in th...
phjTempDF['date'] = pd.to_datetime(phjTempDF['date'], dayfirst = True)

print('This dataset consists of a series of dates, a group ID (1-5) and a random variable. The desired output is to group the data by the group ID and then calculate several statistics for each column of data. So, for example, it would be useful to calculate the mean and SD from the random variable and the earliest and latest dates for each group from the data variable.\n')

with pd.option_context('display.max_rows', 10, 'display.max_columns', 6):
    print(phjTempDF)

date             datetime64[ns]
group                     int64
random_normal           float64
string1                  object
string2                  object
dtype: object
This dataset consists of a series of dates, a group ID (1-5) and a random variable. The desired output is to group the data by the group ID and then calculate several statistics for each column of data. So, for example, it would be useful to calculate the mean and SD from the random variable and the earliest and latest dates for each group from the data variable.

         date  group  random_normal string1 string2
0  2016-01-01      4       4.691670       a       A
1  2016-01-02      5       3.889511       b       B
2  2016-01-03      5       7.188339       c       C
3  2016-01-04      1       2.288360       d       D
4  2016-01-05      2       4.794914       e       E
..        ...    ...            ...     ...     ...
26 2016-01-27      1       4.089191       a       A
27 2016-01-28      2       4.944877       b

#### We could calculate just a single parameter from each variable...

In [3]:
phjTempGroupbyDF = phjTempDF.groupby('group').agg({'date': np.min,
                                                   'random_normal': np.mean})

phjTempGroupbyDF = phjTempGroupbyDF.reset_index(drop=False)

print(phjTempGroupbyDF)

   group  random_normal       date
0      1       4.378687 2016-01-04
1      2       5.594723 2016-01-05
2      3       4.770124 2016-01-08
3      4       4.381841 2016-01-01
4      5       5.412843 2016-01-02


#### We can also concatenate strings

In [4]:
phjTempGroupbyDF = phjTempDF.groupby('group').agg({'date': np.min,
                                                   'random_normal': np.mean,
                                                   'string1': lambda x: ' / '.join(x),
                                                   'string2': lambda x: ' - '.join(x)})

phjTempGroupbyDF = phjTempGroupbyDF.reset_index(drop=False)

print(phjTempGroupbyDF)

   group  random_normal                    string1                    string2  \
0      1       4.378687  d / n / o / q / t / a / d  D - N - O - Q - T - A - D   
1      2       5.594723          e / f / g / r / b          E - F - G - R - B   
2      3       4.770124      h / l / m / s / w / x      H - L - M - S - W - X   
3      4       4.381841  a / i / u / y / z / c / e  A - I - U - Y - Z - C - E   
4      5       5.412843      b / c / j / k / p / v      B - C - J - K - P - V   

        date  
0 2016-01-04  
1 2016-01-05  
2 2016-01-08  
3 2016-01-01  
4 2016-01-02  


#### When there are dates within groups and we want the first or last of those dates, it's a good idea to sort_values on grouping variable and dates before doing groupby. (This was an answer to one of my StackOverflow questions.)

In [5]:
phjTempGroupbyDF = phjTempDF.sort_values(['group','date']).groupby('group').agg({'date': 'first',
                                                                                 'random_normal': np.mean,
                                                                                 'string1': lambda x: ' / '.join(x),
                                                                                 'string2': lambda x: ' - '.join(x)})

phjTempGroupbyDF = phjTempGroupbyDF.reset_index(drop=False)

print(phjTempGroupbyDF)

   group  random_normal                    string1                    string2  \
0      1       4.378687  d / n / o / q / t / a / d  D - N - O - Q - T - A - D   
1      2       5.594723          e / f / g / r / b          E - F - G - R - B   
2      3       4.770124      h / l / m / s / w / x      H - L - M - S - W - X   
3      4       4.381841  a / i / u / y / z / c / e  A - I - U - Y - Z - C - E   
4      5       5.412843      b / c / j / k / p / v      B - C - J - K - P - V   

        date  
0 2016-01-04  
1 2016-01-05  
2 2016-01-08  
3 2016-01-01  
4 2016-01-02  


#### However, may need multiple parameters to be calculated per variable...

In [6]:
phjTempGroupbyDF = phjTempDF.groupby('group').agg({'date': ["first", "last"],
                                                   'random_normal': [np.mean, np.std, "max", np.max, "count"],
                                                   'string1': lambda x: ' / '.join(x),
                                                   'string2': lambda x: ' - '.join(x)})

phjTempGroupbyDF = phjTempGroupbyDF.reset_index(drop=False)

print(phjTempGroupbyDF)

print('\n')

print("In this case, a dataframe with a multiple column index is produced. In order to flatten the column index, we can use a solution posted by Anday Hayden on Stack Overflow on Jan 24 2013 at 18:37 (see http://stackoverflow.com/questions/14507794/python-pandas-how-to-flatten-a-hierarchical-index-in-columns/14508355#14508355). Namely, use:\n   df.columns = [' '.join(col).strip() for col in df.columns.values]")

  group random_normal                                      \
                 mean       std       max      amax count   
0     1      4.378687  1.357445  6.318777  6.318777     7   
1     2      5.594723  1.327565  7.264559  7.264559     5   
2     3      4.770124  1.470794  6.801844  6.801844     6   
3     4      4.381841  0.826034  5.292944  5.292944     7   
4     5      5.412843  1.784498  7.748272  7.748272     6   

                     string1                    string2       date             
                    <lambda>                   <lambda>      first       last  
0  d / n / o / q / t / a / d  D - N - O - Q - T - A - D 2016-01-04 2016-01-30  
1          e / f / g / r / b          E - F - G - R - B 2016-01-05 2016-01-28  
2      h / l / m / s / w / x      H - L - M - S - W - X 2016-01-08 2016-01-24  
3  a / i / u / y / z / c / e  A - I - U - Y - Z - C - E 2016-01-01 2016-01-31  
4      b / c / j / k / p / v      B - C - J - K - P - V 2016-01-02 2016-01-22  


In this ca

In [7]:
phjTempGroupbyDF.columns = ['_'.join(col).strip() for col in phjTempGroupbyDF.columns.values]

print(phjTempGroupbyDF)

   group_  random_normal_mean  random_normal_std  random_normal_max  \
0       1            4.378687           1.357445           6.318777   
1       2            5.594723           1.327565           7.264559   
2       3            4.770124           1.470794           6.801844   
3       4            4.381841           0.826034           5.292944   
4       5            5.412843           1.784498           7.748272   

   random_normal_amax  random_normal_count           string1_<lambda>  \
0            6.318777                    7  d / n / o / q / t / a / d   
1            7.264559                    5          e / f / g / r / b   
2            6.801844                    6      h / l / m / s / w / x   
3            5.292944                    7  a / i / u / y / z / c / e   
4            7.748272                    6      b / c / j / k / p / v   

            string2_<lambda> date_first  date_last  
0  D - N - O - Q - T - A - D 2016-01-04 2016-01-30  
1          E - F - G - R - B

#### But we may need to rename some columns...

In [8]:
phjTempGroupbyDF = phjTempGroupbyDF.rename(columns = {'group_': 'group',
                                                     'date_first': 'earliest_date',
                                                     'date_last': 'latest_date',
                                                     'random_normal_mean': 'mean',
                                                     'random_normal_std': 'sd',
                                                     'random_normal_max': 'max',
                                                     'random_normal_min': 'min',
                                                     'random_normal_amax': 'max2',
                                                     'random_normal_count': 'count',
                                                     'string1_<lambda>': 'string1_concat',
                                                     'string2_<lambda>': 'string2_concat'})

print(phjTempGroupbyDF)

   group      mean        sd       max      max2  count  \
0      1  4.378687  1.357445  6.318777  6.318777      7   
1      2  5.594723  1.327565  7.264559  7.264559      5   
2      3  4.770124  1.470794  6.801844  6.801844      6   
3      4  4.381841  0.826034  5.292944  5.292944      7   
4      5  5.412843  1.784498  7.748272  7.748272      6   

              string1_concat             string2_concat earliest_date  \
0  d / n / o / q / t / a / d  D - N - O - Q - T - A - D    2016-01-04   
1          e / f / g / r / b          E - F - G - R - B    2016-01-05   
2      h / l / m / s / w / x      H - L - M - S - W - X    2016-01-08   
3  a / i / u / y / z / c / e  A - I - U - Y - Z - C - E    2016-01-01   
4      b / c / j / k / p / v      B - C - J - K - P - V    2016-01-02   

  latest_date  
0  2016-01-30  
1  2016-01-28  
2  2016-01-24  
3  2016-01-31  
4  2016-01-22  


#### Can this be done with an ordered dict?

In [9]:
import collections

# To convert the database to one with no repeated measures, we
# use groupby to group by a variable and then use agg() to aggregate on the
# variables we want to keep.
# So, we could create a dict of variables and methods that we want to keep. However,
# Pyton dicts do not keep a track of the order. As a result, the order of variables
# in the final dataframe would be completely different from the original. We could overcome
# this using df = df['a','b','c'] to rearrange the columns but this would mean
# creating a separate list of variable names.
#
# So instead, we can use an OrderedDict. To do this, we need to import collections.
# Then create an ordered dict as follows and pass it to the agg() function as shown.

phjAggOrderedDict = collections.OrderedDict()
phjAggOrderedDict['date'] = ["first","last"]
phjAggOrderedDict['random_normal'] = [np.mean, np.std, "max", np.max, "count"]
phjAggOrderedDict['string1'] = lambda x: ' / '.join(x)
phjAggOrderedDict['string2'] = lambda x: ' / '.join(x)

print(phjAggOrderedDict)

# The following command initially sorts the dataframe by animalID and consultDate
# to ensure that all consultations for each animal are sorted by date. Then the
# dataframe is grouped by animalID and the various columns are aggregated as
# defined in the ordered dictionary.
# As of Pandas 0.17.0, sort() has been deprecated; use sort_values() instead.
phjTempGroupbyDF = phjTempDF.sort_values(['date']).groupby('group').agg(phjAggOrderedDict).reset_index()

print("\n")
print(phjTempGroupbyDF)

OrderedDict([('date', ['first', 'last']), ('random_normal', [<function mean at 0x1042b9950>, <function std at 0x1042b99d8>, 'max', <function amax at 0x1042b9400>, 'count']), ('string1', <function <lambda> at 0x105d989d8>), ('string2', <function <lambda> at 0x105d98f28>)])


  group       date            random_normal                                \
             first       last          mean       std       max      amax   
0     1 2016-01-04 2016-01-30      4.378687  1.357445  6.318777  6.318777   
1     2 2016-01-05 2016-01-28      5.594723  1.327565  7.264559  7.264559   
2     3 2016-01-08 2016-01-24      4.770124  1.470794  6.801844  6.801844   
3     4 2016-01-01 2016-01-31      4.381841  0.826034  5.292944  5.292944   
4     5 2016-01-02 2016-01-22      5.412843  1.784498  7.748272  7.748272   

                           string1                    string2  
  count                   <lambda>                   <lambda>  
0     7  d / n / o / q / t / a / d  D / N / O / Q / T / A

#### But what happens if there is a missing value (either NaN or None)? All hell breaks loose, that's what!

In [10]:
phjTempDF.loc[phjTempDF['date']=='2016-01-02','string1']=None
phjTempDF.loc[phjTempDF['date']=='2016-01-13','string2']=np.nan
print(phjTempDF)

         date  group  random_normal string1 string2
0  2016-01-01      4       4.691670       a       A
1  2016-01-02      5       3.889511    None       B
2  2016-01-03      5       7.188339       c       C
3  2016-01-04      1       2.288360       d       D
4  2016-01-05      2       4.794914       e       E
5  2016-01-06      2       6.744620       f       F
6  2016-01-07      2       7.264559       g       G
7  2016-01-08      3       2.892246       h       H
8  2016-01-09      4       2.862623       i       I
9  2016-01-10      5       7.748272       j       J
10 2016-01-11      5       3.193178       k       K
11 2016-01-12      3       3.703031       l       L
12 2016-01-13      3       5.616434       m     NaN
13 2016-01-14      1       6.318777       n       N
14 2016-01-15      1       5.552102       o       O
15 2016-01-16      5       5.243227       p       P
16 2016-01-17      1       3.213087       q       Q
17 2016-01-18      2       4.224647       r       R
18 2016-01-1

In [11]:
phjAggOrderedDict = collections.OrderedDict()
phjAggOrderedDict['date'] = ["first","last"]
phjAggOrderedDict['random_normal'] = [np.mean, np.std, "max", np.max, "count"]
phjAggOrderedDict['string1'] = lambda x: ' - '.join(x)
phjAggOrderedDict['string2'] = lambda x: ' / '.join(x)

phjTempGroupbyDF = phjTempDF.sort_values(['date']).groupby('group').agg(phjAggOrderedDict).reset_index()

print(phjTempGroupbyDF)

TypeError: sequence item 0: expected str instance, NoneType found

#### The above errors indicate that the presence of NaN or None values causes the join to fail. Instead, converting the missing values to something else before running join avoids this issue.

In [12]:
phjAggOrderedDict = collections.OrderedDict()
phjAggOrderedDict['date'] = ["first","last"]
phjAggOrderedDict['random_normal'] = [np.mean, np.std, "max", np.max, "count"]
phjAggOrderedDict['string1'] = lambda x: ' - '.join(x.fillna('EMPTY'))
phjAggOrderedDict['string2'] = lambda x: ' / '.join(x.fillna('EMPTY'))

phjTempGroupbyDF = phjTempDF.sort_values(['date']).groupby('group').agg(phjAggOrderedDict).reset_index()

print(phjTempGroupbyDF)

  group       date            random_normal                                \
             first       last          mean       std       max      amax   
0     1 2016-01-04 2016-01-30      4.378687  1.357445  6.318777  6.318777   
1     2 2016-01-05 2016-01-28      5.594723  1.327565  7.264559  7.264559   
2     3 2016-01-08 2016-01-24      4.770124  1.470794  6.801844  6.801844   
3     4 2016-01-01 2016-01-31      4.381841  0.826034  5.292944  5.292944   
4     5 2016-01-02 2016-01-22      5.412843  1.784498  7.748272  7.748272   

                           string1                    string2  
  count                   <lambda>                   <lambda>  
0     7  d - n - o - q - t - a - d  D / N / O / Q / T / A / D  
1     5          e - f - g - r - b          E / F / G / R / B  
2     6      h - l - m - s - w - x  H / L / EMPTY / S / W / X  
3     7  a - i - u - y - z - c - e  A / I / U / Y / Z / C / E  
4     6  EMPTY - c - j - k - p - v      B / C / J / K / P / V  
